In [81]:
# Download zip dataset from Google Drive
! filename='entailment_trees_emnlp2021_data_v3.zip'
! fileid='1kVr-YsUVFisceiIklvpWEe0kHNSIFtNh'
! wget --load-cookies /tmp/cookies.txt "https://drive.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://drive.google.com/uc?export=download&id=${fileid}' -O- | sed -rn 's/.confirm=([0-9A-Za-z_]+)./\1\n/p')&id=${fileid}" -O ${filename} && rm -rf /tmp/cookies.txt
# Unzip
! unzip -q ${filename}
! rm ${filename}

--2022-11-13 04:23:28--  https://drive.google.com/uc?export=download&confirm=&id=1kVr-YsUVFisceiIklvpWEe0kHNSIFtNh
Resolving drive.google.com (drive.google.com)... 142.251.40.110, 2607:f8b0:4006:81f::200e
Connecting to drive.google.com (drive.google.com)|142.251.40.110|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-00-80-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/hnk4a2dn58nhe5mhvc5mnkmbq0mpbqf9/1668313425000/03839794848891343351/*/1kVr-YsUVFisceiIklvpWEe0kHNSIFtNh?e=download&uuid=c5068930-eb38-4e84-ad2b-84f791149a33 [following]
--2022-11-13 04:23:30--  https://doc-00-80-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/hnk4a2dn58nhe5mhvc5mnkmbq0mpbqf9/1668313425000/03839794848891343351/*/1kVr-YsUVFisceiIklvpWEe0kHNSIFtNh?e=download&uuid=c5068930-eb38-4e84-ad2b-84f791149a33
Resolving doc-00-80-docs.googleusercontent.com (doc-00-80-docs.googleusercontent.com)... 142.251.35.161, 2607:f8

In [1]:
import jsonlines
import re

In [41]:
read_path='entailment_trees_emnlp2021_data_v3/dataset/task_1/train.jsonl'
with jsonlines.open(read_path, "r") as rfd:
    for data in rfd:
        for key in data.keys():
            if key in ['context', 'hypothesis','id']:
                print(f"key: {key}")
                print(f"value: {data[key]}\n")
        break
rfd.close()

key: id
value: MDSA_2009_4_30

key: context
value: sent1: leo is a kind of constellation sent2: the earth revolving around the sun causes stars to appear in different areas in the sky at different times of year sent3: a constellation contains stars

key: hypothesis
value: the earth revolving around the sun causes leo to appear in different areas in the sky at different times of year



In [6]:
hypo_dict={}
pattern = r'sent\d+: '
read_path='entailment_trees_emnlp2021_data_v3/dataset/task_1/train.jsonl'
with jsonlines.open(read_path, "r") as rfd:
    for data in rfd:
        result = re.split(pattern, data['context'])
        result_n = set([x.strip() for x in result if x.strip()!=''])
        hypo_dict[data['hypothesis']]=result_n
rfd.close()

In [8]:
for i,hypothesis1 in enumerate(hypo_dict.keys()):
    print(hypo_dict[hypothesis1])
    if i>10:
        break

{'the earth revolving around the sun causes stars to appear in different areas in the sky at different times of year', 'leo is a kind of constellation', 'a constellation contains stars'}
{'a star is a kind of celestial object / celestial body', 'earth is a kind of celestial object', 'stars appear to move relative to the horizon during the night', 'the earth rotating on its axis causes stars to appear to move across the sky at night', "apparent motion is when an object appears to move relative to another object 's position"}
{"diurnal motion is when objects in the sky appear to move due to earth 's rotation on its axis", 'the earth rotating on its axis causes stars / the moon to appear to move across the sky at night', 'stars appear to move relative to the horizon during the night'}
{'a source of something produces that something', 'as a source of light becomes closer , the light will appear brighter', 'the sun is the star that is closest to earth', 'a star produces light'}
{'the sun is

In [35]:
joint_cnt=0
joint_hypo=set()
joint_hypo_pair=[]
for i,hypothesis1 in enumerate(hypo_dict.keys()):
    for j,hypothesis2 in enumerate(hypo_dict.keys()):
        if j<=i:
            continue
        if not hypo_dict[hypothesis2].isdisjoint(hypo_dict[hypothesis1]):
            print(hypo_dict[hypothesis1])
            print(hypo_dict[hypothesis2])
            joint_hypo.add(hypothesis1)
            joint_hypo.add(hypothesis2)
            joint_hypo_pair.append((hypothesis1,hypothesis2))
            print("--------------------------------------------------")
            joint_cnt+=1
    

{'a star is a kind of celestial object / celestial body', 'earth is a kind of celestial object', 'stars appear to move relative to the horizon during the night', 'the earth rotating on its axis causes stars to appear to move across the sky at night', "apparent motion is when an object appears to move relative to another object 's position"}
{"diurnal motion is when objects in the sky appear to move due to earth 's rotation on its axis", 'the earth rotating on its axis causes stars / the moon to appear to move across the sky at night', 'stars appear to move relative to the horizon during the night'}
--------------------------------------------------
{'a star is a kind of celestial object / celestial body', 'earth is a kind of celestial object', 'stars appear to move relative to the horizon during the night', 'the earth rotating on its axis causes stars to appear to move across the sky at night', "apparent motion is when an object appears to move relative to another object 's position"}


{'a bird is a kind of animal', 'an animal is a kind of living thing', 'a seed is a kind of young plant', 'a rabbit is a kind of animal', 'a plant is a kind of living thing'}
{'a bird is a kind of animal', 'a butterfly is a kind of animal', 'birds eat butterflies', 'a predator avoiding some types of prey is a learned behavior', 'birds avoid a specific kind of butterfly', 'learned behaviors are learned over the lifetime of the animal', 'if an animal is eaten by other animals , then the animal will be the prey', 'if an animal eats other animals for food , then the animal will be the predator'}
--------------------------------------------------
{'a bird is a kind of animal', 'an animal is a kind of living thing', 'a seed is a kind of young plant', 'a rabbit is a kind of animal', 'a plant is a kind of living thing'}
{'an animal is a kind of living thing', 'genes are able to determine the inherited characteristics of a living thing', 'the growth of antler is an inherited characteristic', 'wh

In [36]:
print(joint_cnt)

3790


In [37]:
print(len(joint_hypo))

996


In [38]:
print(len(hypo_dict.keys()))

1276


In [39]:
for pair in joint_hypo_pair:
    print(pair)

('the earth rotating on its axis causes stars to move relative to the horizon during the night', 'the earth rotating on its axis causes stars to appear to move relative to the horizon during the night')
('the earth rotating on its axis causes stars to move relative to the horizon during the night', 'the sun will appear larger than other stars because it is the closest star to earth')
('the earth rotating on its axis causes stars to move relative to the horizon during the night', 'the sun is the celestial object that produces light and heat and is closest to earth')
('the earth rotating on its axis causes stars to move relative to the horizon during the night', 'a star is an astronomical body that emits light through nuclear reactions')
('the earth rotating on its axis causes stars to move relative to the horizon during the night', 'the rocket will weigh less but have the same mass on the moon than on earth')
('the earth rotating on its axis causes stars to move relative to the horizon 

In [43]:
congruent_cnt=0
congruent_hypo=set()
congruent_hypo_pair=[]
for i,hypothesis1 in enumerate(hypo_dict.keys()):
    for j,hypothesis2 in enumerate(hypo_dict.keys()):
        if j<=i:
            continue
        if hypo_dict[hypothesis2]==hypo_dict[hypothesis1]:
            print(hypo_dict[hypothesis1])
            print(hypo_dict[hypothesis2])
            congruent_hypo.add(hypothesis1)
            congruent_hypo.add(hypothesis2)
            congruent_hypo_pair.append((hypothesis1,hypothesis2))
            print("--------------------------------------------------")
            congruent_cnt+=1

{'a planet rotating causes cycles of day and night on that planet', 'the earth rotates on its tilted axis', 'earth is a kind of planet'}
{'a planet rotating causes cycles of day and night on that planet', 'the earth rotates on its tilted axis', 'earth is a kind of planet'}
--------------------------------------------------
{'gravity causes orbits', 'planets in the solar system orbit the sun'}
{'gravity causes orbits', 'planets in the solar system orbit the sun'}
--------------------------------------------------
{'mechanical weathering means breaking down rocks from a larger whole into smaller pieces by mechanical means', 'ice wedging is a kind of mechanical weathering', 'cycles of freezing and thawing water cause ice wedging'}
{'mechanical weathering means breaking down rocks from a larger whole into smaller pieces by mechanical means', 'ice wedging is a kind of mechanical weathering', 'cycles of freezing and thawing water cause ice wedging'}
------------------------------------------

In [44]:
for pair in congruent_hypo_pair:
    print(pair)

('earth rotating on its axis causes cycles of day and night on earth', 'earth rotating on its axis causes the cycle of day and night on earth')
('gravity causes the planets in the solar system to orbit the sun', 'gravity causes planets in the solar system to orbit the sun')
('rocks will be broken down from a larger whole into smaller pieces by cycles of freezing and thawing water', 'cycles of freezing and thawing water will break down rocks from larger whole into smaller pieces')
('rocks will be broken down from a larger whole into smaller pieces by cycles of freezing and thawing water', 'cycles of freezing and thawing water cause the rocks to break into smaller pieces')
('cycles of freezing and thawing water will break down rocks from larger whole into smaller pieces', 'cycles of freezing and thawing water cause the rocks to break into smaller pieces')
('when a flashlight is used, the energy is changed from chemical energy into light energy', 'when a flashlight is used, the energy is 

In [13]:
print(congruent_cnt)
print(len(congruent_hypo))

7
11
